In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from skimpy import clean_columns, skim
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


Downloading the data    

In [2]:
brent = clean_columns (pd.read_csv("brent.csv", parse_dates= ['Tarih']))
cash_flow_train = clean_columns(pd.read_csv("cash_flow_train.csv", parse_dates= ['Date']))  
depo_pump_imm =  clean_columns(pd.read_csv("depo_pump_imm.csv")) 
otv =  clean_columns(pd.read_csv("otv.csv", parse_dates= ['Tarih'])) 
platts = clean_columns(pd.read_csv("platts.csv", parse_dates= ['Tarih']))  
sample_submission = pd.read_csv("sample_submission.csv")
usd =  clean_columns(pd.read_csv("usd.csv", parse_dates= ['Tarih'])) 
volume = clean_columns(pd.read_csv("volume.csv", parse_dates= ['Posting date']))

7 column names have been cleaned

17 column names have been cleaned

5 column names have been cleaned

5 column names have been cleaned

7 column names have been cleaned

8 column names have been cleaned

5 column names have been cleaned

In [3]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Date                          42 non-null     object
 1   Net Cashflow from Operations  42 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 800.0+ bytes


In [4]:
cash_flow_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            773 non-null    datetime64[ns]
 1   total_inflows                   773 non-null    float64       
 2   customers_dds                   773 non-null    int64         
 3   customers_eft                   773 non-null    float64       
 4   t_&_s_collections               773 non-null    float64       
 5   fx_sales                        773 non-null    float64       
 6   other_operations                773 non-null    float64       
 7   total_outflows                  773 non-null    float64       
 8   tupras                          773 non-null    float64       
 9   other_oil                       773 non-null    float64       
 10  gas                             773 non-null    float64       
 11  import

In [5]:
cash_flow_train.describe

<bound method NDFrame.describe of           date  total_inflows  customers_dds  customers_eft  \
0   2019-01-02   1.747050e+07        7718750   9.655500e+06   
1   2019-01-03   1.153048e+07        3431500   8.098978e+06   
2   2019-01-04   2.591328e+07       13775750   1.213753e+07   
3   2019-01-07   5.259345e+07       39041750   1.355170e+07   
4   2019-01-08   3.650429e+07       15137500   2.136679e+07   
..         ...            ...            ...            ...   
768 2023-01-26   1.408741e+08       90239000   4.977288e+07   
769 2023-01-27   1.062562e+08       53296500   5.282375e+07   
770 2023-01-30   5.225381e+08      350467750   1.679393e+08   
771 2023-01-31   2.323393e+08      131395500   9.922482e+07   
772 2023-02-01   9.117482e+07       56928500   3.354297e+07   

     t_&_s_collections  fx_sales  other_operations  total_outflows  \
0         0.000000e+00       0.0           96250.0   -4.631593e+07   
1         0.000000e+00       0.0               0.0   -1.535349e+07   

In [6]:
brent.describe

<bound method NDFrame.describe of           tarih       urun avrupa_birligi_para_birimi  ab_piyasa_fiyat  \
0    2023-02-01  ICE BRENT                      $/BBL            82.84   
1    2023-01-31  ICE BRENT                      $/BBL            84.49   
2    2023-01-30  ICE BRENT                      $/BBL            84.90   
3    2023-01-29  ICE BRENT                      $/BBL            86.66   
4    2023-01-28  ICE BRENT                      $/BBL            86.66   
...         ...        ...                        ...              ...   
1852 2018-01-06  ICE BRENT                      $/BBL            67.62   
1853 2018-01-05  ICE BRENT                      $/BBL            67.62   
1854 2018-01-04  ICE BRENT                      $/BBL            68.07   
1855 2018-01-03  ICE BRENT                      $/BBL            67.84   
1856 2018-01-02  ICE BRENT                      $/BBL            66.57   

      ab_piyasa_fiyat_yuksek  ab_piyasa_fiyat_dusuk  dolar_kuru_sats  
0     

In [7]:
brent_cash = cash_flow_train.merge(brent, left_on= 'date', right_on='tarih', how= 'left').drop('tarih', axis =1)


In [8]:
dates = pd.DataFrame({'date' :pd.date_range(start='1-1-2018', end= '31-12-2022')})  
dates['year'] = dates['date'].dt.year
dates['month'] = dates['date'].dt.month

/var/folders/t1/db5m68vn10v3148_nq_7r8b80000gn/T/ipykernel_1877/870535568.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dates = pd.DataFrame({'date' :pd.date_range(start='1-1-2018', end= '31-12-2022')})


In [9]:
depo = dates.merge(depo_pump_imm, left_on= ['year', 'month'], right_on= ['yl', 'ay']).drop(['yl','ay'], axis = 1)

In [10]:
brent_cash_depo = brent_cash.merge(depo, on = 'date', how = 'left')

In [11]:
otv.head()

,tarih,otv,adet,otv_oran,otv_uygulanan_urun_ad
0,2018-05-22,2124.10,1000,2.12410,Kursunsuz Benzin 95
1,2018-05-22,2124.10,1000,2.12410,Kursunsuz Benzin 95
2,2018-05-22,2081.62,1000,2.08162,Etanollü Kurşunsuz Benzin 95
3,2018-05-22,2081.62,1000,2.08162,Etanollü Kurşunsuz Benzin 95
4,2018-05-22,1567.90,1000,1.56790,Motorin


In [12]:
otv.drop_duplicates(inplace= True)

In [13]:
otv_fixed =  otv.drop(['otv','adet'], axis = 1).pivot(index= 'tarih', columns= 'otv_uygulanan_urun_ad').reset_index()

In [14]:
otv_fixed = otv_fixed.droplevel(0, axis =1)

In [15]:
otv_fixed.columns = ['date', 'benzinli_motorin', 'etanallu_kursunsuz_benzin_95', 'kursunsuz_benzin_95','lpg','motorin']

In [16]:
otv_fixed.fillna(method= 'bfill').fillna(method= 'ffill', inplace= True )

In [17]:
brent_cash_depo_otv =  brent_cash_depo.merge(otv_fixed,left_on = 'date', right_on = 'date', how = 'left')

In [18]:
brent_cash_depo_otv.head()

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,year,month,imm_tl_m_3,pump_price,depot_price,benzinli_motorin,etanallu_kursunsuz_benzin_95,kursunsuz_benzin_95,lpg,motorin
0,2019-01-02,1.747050e+07,7718750,9.655500e+06,0.0,0.0,96250.0,-4.631593e+07,-1.457022e+07,-144151.00,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
1,2019-01-03,1.153048e+07,3431500,8.098978e+06,0.0,0.0,0.0,-1.535349e+07,-1.337181e+07,-300799.25,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
2,2019-01-04,2.591328e+07,13775750,1.213753e+07,0.0,0.0,0.0,-1.098651e+08,-2.459390e+07,0.00,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
3,2019-01-07,5.259345e+07,39041750,1.355170e+07,0.0,0.0,0.0,-5.165489e+07,-3.110594e+07,-3364373.00,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN
4,2019-01-08,3.650429e+07,15137500,2.136679e+07,0.0,0.0,0.0,-1.842178e+07,-1.246099e+07,-16765.25,...,2019.0,1.0,"723,05","6,02","4.784,03",NaN,NaN,NaN,NaN,NaN


In [19]:
brent_cash_depo_otv_platts =  brent_cash_depo_otv.merge(platts,left_on = 'date', right_on = 'tarih', how = 'left')

In [20]:
brent_cash_depo_otv_platts_usd =  brent_cash_depo_otv_platts.merge(usd,left_on = 'date', right_on = 'tarih', how = 'left')

In [21]:
skim(brent_cash_depo_otv_platts_usd)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1523   │ │ float64     │ 36    │                                                          │
│ │ Number of columns │ 48     │ │ string      │ 7     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 3     │                                                          │
│                                │ int64       │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_nam ┃      ┃      ┃           ┃           ┃            ┃           ┃           ┃           ┃        ┃  │
│ ┃ e          ┃ NA   ┃ NA % ┃ mean      ┃ sd        ┃ p0         ┃ p25       ┃ p75       ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ total_infl │    0 │    0 │ 120000000 │ 120000000 │   12000000 │  43000000 │ 150000000 │ 860000000 │   █▁   │  │
│ │ ows        │      │      │           │           │            │           │           │           │        │  │
│ │ customers_ │    0 │    0 │  56000000 │  64000000 │    2500000 │  17000000 │  70000000 │ 570000000 │   █▁   │  │
│ │ dds        │      │      │           │           │            │           │           │           │        │  │
│ │ customers_ │    0 │    0 │  44000000 │  44000000 │    6200000 │  18000000 │  58000000 │ 330000000 │   █▂   │  │
│ │ eft        │      │      │           │           │            │           │           │           │        │  │
│ │ t_&_s_coll │    0 │    0 │  12000000 │  10000000 │          0 │   5500000 │  15000000 │  79000000 │   █▃   │  │
│ │ ections    │      │      │           │           │            │           │           │           │        │  │
│ │ fx_sales   │    0 │    0 │    410000 │   4400000 │          0 │         0 │         0 │ 140000000 │   █    │  │
│ │ other_oper │    0 │    0 │   2700000 │   9800000 │          0 │         0 │         0 │ 100000000 │   █    │  │
│ │ ations     │      │      │           │           │            │           │           │           │        │  │
│ │ total_outf │    0 │    0 │ -10000000 │  95000000 │ -590000000 │ -14000000 │ -33000000 │  -6800000 │    ▁▃█ │  │
│ │ lows       │      │      │         0 │           │            │         0 │           │           │        │  │
│ │ tupras     │    0 │    0 │ -39000000 │  43000000 │ -300000000 │ -51000000 │ -12000000 │   -940000 │     ▂█ │  │
│ │ other_oil  │    0 │    0 │ -16000000 │  26000000 │ -210000000 │ -17000000 │  -3100000 │         0 │     ▁█ │  │
│ │ gas        │    0 │    0 │  -5100000 │   6300000 │  -45000000 │  -6400000 │  -1200000 │         0 │     ▁█ │  │
│ │ import_pay │    0 │    0 │ -20000000 │  29000000 │ -320000000 │ -31000000 │         0 │         0 │     ▁█ │  │
│ │ ments_fx_p │      │      │           │           │            │           │           │           │        │  │
│ │ tax        │    0 │    0 │  -8600000 │  16000000 │ -130000000 │  -7700000 │   -230000 │         0 │     ▁█ │  │
│ │ operationa │    0 │    0 │  -4600000 │  12000000 │ -

In [22]:
brent_cash_depo_otv_platts_usd.drop(['tarih_x', 'tarih_y'], axis= 1, inplace= True)

In [23]:
volume.head()

,posting_date,product,material_description,category,sales_volumes_in_l_15
0,2019-01-01,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,935955.0000
1,2019-01-01,400003769,SH ULG 95 10ppmS E0 Udy Mkd FuelSave TR,Shell Fleet Solutions 100018,82.0275
2,2019-01-01,400003769,SH ULG 95 10ppmS E0 Udy Mkd FuelSave TR,Undercanopy 800001,2268.0725
3,2019-01-01,400003981,AGO 10ppmS B0 Udy Mkd FuelSave Diesel TR,Shell Fleet Solutions 100018,4625.2275
4,2019-01-01,400003981,AGO 10ppmS B0 Udy Mkd FuelSave Diesel TR,Undercanopy 800001,39751.6400


In [24]:
volume['material_description'].unique()

array(['BF AGO 10ppmS B0 Udy Mkd Turkey',
       'SH ULG 95 10ppmS E0 Udy Mkd FuelSave TR',
       'AGO 10ppmS B0 Udy Mkd FuelSave Diesel TR', ...,
       'AC Paroil E_1*209L_A1MP', 'Alvania 0860_1*180kg_A1P5',
       'HlxUltProfAO-L 0W20_3*5L_EURO'], dtype=object)

In [25]:
volume['category'].unique()

array(['Supply 310001', 'Shell Fleet Solutions 100018',
       'Undercanopy 800001', 'Lubricants 100034',
       'Commercial Fuels 800006', 'LSC 600010'], dtype=object)

In [26]:
volume[(volume['material_description'] == 'BF AGO 10ppmS B0 Udy Mkd Turkey' )& (volume['category'] =='Supply 310001')]

,posting_date,product,material_description,category,sales_volumes_in_l_15
0,2019-01-01,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,9.359550e+05
24,2019-01-02,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,4.189095e+05
157,2019-01-03,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2.828942e+05
294,2019-01-04,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,7.057548e+05
400,2019-01-05,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,4.236176e+06
...,...,...,...,...,...
112954,2023-01-27,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2.009900e+04
113094,2023-01-28,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,5.386850e+04
113116,2023-01-30,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,7.152808e+05
113240,2023-01-31,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,9.914816e+04


In [27]:
d = {'a' : volume['material_description'].unique(), 'b' :volume['category'].unique() }

In [28]:
grid =  pd.MultiIndex.from_product(d.values(),names=d.keys()).to_frame().reset_index(drop=True)

In [29]:
grid

,a,b
0,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001
1,BF AGO 10ppmS B0 Udy Mkd Turkey,Shell Fleet Solutions 100018
2,BF AGO 10ppmS B0 Udy Mkd Turkey,Undercanopy 800001
3,BF AGO 10ppmS B0 Udy Mkd Turkey,Lubricants 100034
4,BF AGO 10ppmS B0 Udy Mkd Turkey,Commercial Fuels 800006
...,...,...
6529,HlxUltProfAO-L 0W20_3*5L_EURO,Shell Fleet Solutions 100018
6530,HlxUltProfAO-L 0W20_3*5L_EURO,Undercanopy 800001
6531,HlxUltProfAO-L 0W20_3*5L_EURO,Lubricants 100034
6532,HlxUltProfAO-L 0W20_3*5L_EURO,Commercial Fuels 800006


In [30]:
grid.merge(volume, left_on= ['a', 'b'], right_on= ['material_description', 'category'])

,a,b,posting_date,product,material_description,category,sales_volumes_in_l_15
0,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-01,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,935955.00
1,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-02,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,418909.50
2,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-03,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,282894.25
3,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-04,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,705754.75
4,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,2019-01-05,400003756,BF AGO 10ppmS B0 Udy Mkd Turkey,Supply 310001,4236176.00
...,...,...,...,...,...,...,...
113492,Advance SX2Predl_12*1L_A926,LSC 600010,2023-01-11,550069610,Advance SX2Predl_12*1L_A926,LSC 600010,15.00
113493,AC Paroil E_1*209L_A1MP,Lubricants 100034,2022-12-08,550058411,AC Paroil E_1*209L_A1MP,Lubricants 100034,418.00
113494,Alvania 0860_1*180kg_A1P5,LSC 600010,2022-12-16,550069409,Alvania 0860_1*180kg_A1P5,LSC 600010,4050.00
113495,HlxUltProfAO-L 0W20_3*5L_EURO,Lubricants 100034,2023-01-09,550069004,HlxUltProfAO-L 0W20_3*5L_EURO,Lubricants 100034,7.50


In [31]:
volume_fixed =  volume.pivot( columns= ['product', 'material_description', 'category'], values=  'sales_volumes_in_l_15', index= 'posting_date')

In [32]:
volume_fixed = volume_fixed.droplevel(level=0, axis='columns')

In [33]:
volume_fixed = volume_fixed.droplevel(level=0, axis='columns')

In [34]:
volume_fixed 

category,Supply 310001,Shell Fleet Solutions 100018,Undercanopy 800001,Shell Fleet Solutions 100018,Undercanopy 800001,Undercanopy 800001,Supply 310001,Shell Fleet Solutions 100018,Undercanopy 800001,Shell Fleet Solutions 100018,...,LSC 600010,LSC 600010,Lubricants 100034,Lubricants 100034,LSC 600010,Lubricants 100034,LSC 600010,Lubricants 100034,LSC 600010,LSC 600010
posting_date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,9.359550e+05,82.0275,2268.0725,4625.2275,39751.6400,3928.75,18565.81525,1368.7725,5.360675e+04,11.4825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,4.189095e+05,NaN,28243.7500,184.2500,265244.2500,248496.50,55268.79800,212.5000,8.376430e+05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,2.828942e+05,13.2500,29391.2500,23.5000,318475.2500,297583.50,55116.98525,32.7500,8.841672e+05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,7.057548e+05,NaN,45345.5000,NaN,423750.7500,304918.25,54648.15150,92.7650,1.129202e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,4.236176e+06,NaN,47572.7500,12016.0000,518194.2500,398006.75,45722.45025,1720.5000,1.358899e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-28,5.386850e+04,NaN,43633.0000,NaN,483387.7500,521717.50,26815.75000,NaN,1.738675e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-29,NaN,NaN,NaN,NaN,NaN,NaN,27003.50000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-30,7.152808e+05,NaN,49582.7500,NaN,513549.7500,485988.00,27683.50000,NaN,1.844408e+06,NaN,...,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
brent_cash_depo_otv_platts_usd_volume=  brent_cash_depo_otv_platts_usd.merge(volume_fixed, left_on= 'date', right_on= 'posting_date', how= 'left')

In [36]:
brent_cash_depo_otv_platts_usd_volume.isnull().sum().sort_values(ascending= False)

Lubricants 100034     1523
LSC 600010            1523
Lubricants 100034     1523
Lubricants 100034     1522
LSC 600010            1522
                      ... 
Undercanopy 800001       0
total_inflows            0
dolar_kuru_sats_x        0
Undercanopy 800001       0
date                     0
Length: 1590, dtype: int64

In [37]:
brent_cash_depo_otv_platts_usd_volume.shape

(1523, 1590)

In [38]:
limit_par = len(brent_cash_depo_otv_platts_usd_volume) * 0.75
brent_cash_depo_otv_platts_usd_volume.dropna(thresh= limit_par, axis=1, inplace= True)

In [39]:
brent_cash_depo_otv_platts_usd_volume.tail()

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Supply 310001,Supply 310001,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034
1518,2023-01-26,1.408741e+08,90239000,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,117220.00,NaN,130759.00,25125.00,NaN,28334.75,NaN,51036.50,8778.0,17423.48850
1519,2023-01-27,1.062562e+08,53296500,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,143861.50,8092.75,263358.50,29175.25,NaN,40980.75,NaN,118578.75,4284.5,19849.46500
1520,2023-01-30,5.225381e+08,350467750,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,270295.00,13165.75,281245.75,24063.00,NaN,95048.00,NaN,243090.25,104.5,13989.79850
1521,2023-01-31,2.323393e+08,131395500,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,126757.25,16877.25,294369.25,36002.75,NaN,92303.25,NaN,241450.00,104.5,NaN
1522,2023-02-01,9.117482e+07,56928500,3.354297e+07,7.033439e+05,0.0,0.0,-2.020503e+08,-9.993562e+07,-3.153884e+05,...,144389.50,3932.25,158910.50,9511.75,NaN,27455.00,NaN,81469.00,2926.0,19731.27625


In [40]:
sample_submission.head()

,Date,Net Cashflow from Operations
0,2023-02-02,0
1,2023-02-03,0
2,2023-02-06,0
3,2023-02-07,0
4,2023-02-08,0


In [41]:
sample_submission.tail()

,Date,Net Cashflow from Operations
37,2023-03-27,0
38,2023-03-28,0
39,2023-03-29,0
40,2023-03-30,0
41,2023-03-31,0


In [42]:
brent_cash_depo_otv_platts_usd_volume['date']

0      2019-01-02
1      2019-01-02
2      2019-01-03
3      2019-01-03
4      2019-01-04
          ...    
1518   2023-01-26
1519   2023-01-27
1520   2023-01-30
1521   2023-01-31
1522   2023-02-01
Name: date, Length: 1523, dtype: datetime64[ns]

In [43]:
brent_cash_depo_otv_platts_usd_volume.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1523 entries, 0 to 1522
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            1523 non-null   datetime64[ns]
 1   total_inflows                   1523 non-null   float64       
 2   customers_dds                   1523 non-null   int64         
 3   customers_eft                   1523 non-null   float64       
 4   t_&_s_collections               1523 non-null   float64       
 5   fx_sales                        1523 non-null   float64       
 6   other_operations                1523 non-null   float64       
 7   total_outflows                  1523 non-null   float64       
 8   tupras                          1523 non-null   float64       
 9   other_oil                       1523 non-null   float64       
 10  gas                             1523 non-null   float64       
 11  impo

In [44]:
brent_cash_depo_otv_platts_usd_volume['date'] + pd.Timedelta(days=58)

0      2019-03-01
1      2019-03-01
2      2019-03-02
3      2019-03-02
4      2019-03-03
          ...    
1518   2023-03-25
1519   2023-03-26
1520   2023-03-29
1521   2023-03-30
1522   2023-03-31
Name: date, Length: 1523, dtype: datetime64[ns]

In [45]:
brent_cash_depo_otv_platts_usd_volume.head()

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Supply 310001,Supply 310001,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034
0,2019-01-02,1.747050e+07,7718750,9.655500e+06,0.0,0.0,96250.0,-4.631593e+07,-1.457022e+07,-144151.00,...,59230.50,10863.75,420089.00,10315.75,80332.25,120440.25,6034.25,560537.00,365.75,NaN
1,2019-01-02,1.747050e+07,7718750,9.655500e+06,0.0,0.0,96250.0,-4.631593e+07,-1.457022e+07,-144151.00,...,59230.50,10863.75,420089.00,10315.75,80332.25,120440.25,6034.25,560537.00,365.75,NaN
2,2019-01-03,1.153048e+07,3431500,8.098978e+06,0.0,0.0,0.0,-1.535349e+07,-1.337181e+07,-300799.25,...,138387.50,4102.00,378238.25,20507.00,75298.75,121302.50,2230.25,586303.50,NaN,NaN
3,2019-01-03,1.153048e+07,3431500,8.098978e+06,0.0,0.0,0.0,-1.535349e+07,-1.337181e+07,-300799.25,...,138387.50,4102.00,378238.25,20507.00,75298.75,121302.50,2230.25,586303.50,NaN,NaN
4,2019-01-04,2.591328e+07,13775750,1.213753e+07,0.0,0.0,0.0,-1.098651e+08,-2.459390e+07,0.00,...,114075.75,11056.25,415215.50,13539.75,39619.00,161824.00,21196.25,689547.25,NaN,NaN


In [46]:
brent_cash_depo_otv_platts_usd_volume.shift(58)

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Supply 310001,Supply 310001,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,2022-12-05,3.902473e+08,207791250.0,1.309120e+08,3.104404e+07,0.0,20500000.0,-3.092551e+08,-1.457957e+08,-6.405236e+07,...,NaN,9850.00,107824.00,16219.00,41010.75,72124.25,10397.25,335255.75,NaN,30828.56425
1519,2022-12-05,3.902473e+08,207791250.0,1.309120e+08,3.104404e+07,0.0,20500000.0,-3.092551e+08,-1.457957e+08,-6.405236e+07,...,NaN,9850.00,107824.00,16219.00,41010.75,72124.25,10397.25,335255.75,NaN,30828.56425
1520,2022-12-06,1.654308e+08,42218250.0,8.107418e+07,4.213842e+07,0.0,0.0,-2.088210e+08,-8.916274e+07,-1.974596e+07,...,9161.75,7435.75,334840.50,33275.75,7886.00,162008.50,19989.00,337446.75,522.50,5816.12350
1521,2022-12-06,1.654308e+08,42218250.0,8.107418e+07,4.213842e+07,0.0,0.0,-2.088210e+08,-8.916274e+07,-1.974596e+07,...,9161.75,7435.75,334840.50,33275.75,7886.00,162008.50,19989.00,337446.75,522.50,5816.12350


In [47]:
brent_cash_depo_otv_platts_usd_volume

,date,total_inflows,customers_dds,customers_eft,t_&_s_collections,fx_sales,other_operations,total_outflows,tupras,other_oil,...,Supply 310001,Supply 310001,Commercial Fuels 800006,Commercial Fuels 800006,Commercial Fuels 800006,Undercanopy 800001,Commercial Fuels 800006,Undercanopy 800001,LSC 600010,Lubricants 100034
0,2019-01-02,1.747050e+07,7718750,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,59230.50,10863.75,420089.00,10315.75,80332.25,120440.25,6034.25,560537.00,365.75,NaN
1,2019-01-02,1.747050e+07,7718750,9.655500e+06,0.000000e+00,0.0,96250.0,-4.631593e+07,-1.457022e+07,-1.441510e+05,...,59230.50,10863.75,420089.00,10315.75,80332.25,120440.25,6034.25,560537.00,365.75,NaN
2,2019-01-03,1.153048e+07,3431500,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,138387.50,4102.00,378238.25,20507.00,75298.75,121302.50,2230.25,586303.50,NaN,NaN
3,2019-01-03,1.153048e+07,3431500,8.098978e+06,0.000000e+00,0.0,0.0,-1.535349e+07,-1.337181e+07,-3.007992e+05,...,138387.50,4102.00,378238.25,20507.00,75298.75,121302.50,2230.25,586303.50,NaN,NaN
4,2019-01-04,2.591328e+07,13775750,1.213753e+07,0.000000e+00,0.0,0.0,-1.098651e+08,-2.459390e+07,0.000000e+00,...,114075.75,11056.25,415215.50,13539.75,39619.00,161824.00,21196.25,689547.25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,2023-01-26,1.408741e+08,90239000,4.977288e+07,8.622500e+05,0.0,0.0,-1.318398e+08,-4.748971e+07,-3.760017e+05,...,117220.00,NaN,130759.00,25125.00,NaN,28334.75,NaN,51036.50,8778.00,17423.48850
1519,2023-01-27,1.062562e+08,53296500,5.282375e+07,1.359018e+05,0.0,0.0,-2.777992e+08,-4.800505e+07,-1.360219e+08,...,143861.50,8092.75,263358.50,29175.25,NaN,40980.75,NaN,118578.75,4284.50,19849.46500
1520,2023-01-30,5.225381e+08,350467750,1.679393e+08,4.131052e+06,0.0,0.0,-3.223933e+08,-1.854239e+08,-2.196258e+07,...,270295.00,13165.75,281245.75,24063.00,NaN,95048.00,NaN,243090.25,104.50,13989.79850
1521,2023-01-31,2.323393e+08,131395500,9.922482e+07,1.718965e+06,0.0,0.0,-1.187000e+08,-3.198080e+07,-1.251479e+06,...,126757.25,16877.25,294369.25,36002.75,NaN,92303.25,NaN,241450.00,104.50,NaN


In [48]:
brent_cash_depo_otv_platts_usd_volume['date_'] = brent_cash_depo_otv_platts_usd_volume['date'] + pd.Timedelta(days=58)

In [49]:
brent_cash_depo_otv_platts_usd_volume[['date','date_']]

,date,date_
0,2019-01-02,2019-03-01
1,2019-01-02,2019-03-01
2,2019-01-03,2019-03-02
3,2019-01-03,2019-03-02
4,2019-01-04,2019-03-03
...,...,...
1518,2023-01-26,2023-03-25
1519,2023-01-27,2023-03-26
1520,2023-01-30,2023-03-29
1521,2023-01-31,2023-03-30


In [50]:
brent_cash_depo_otv_platts_usd_volume.set_index('date_', inplace= True)

In [51]:
sample_submission.tail()

,Date,Net Cashflow from Operations
37,2023-03-27,0
38,2023-03-28,0
39,2023-03-29,0
40,2023-03-30,0
41,2023-03-31,0


In [52]:
train = brent_cash_depo_otv_platts_usd_volume['2019-03-01' : '2023-02-01']

In [53]:
test = brent_cash_depo_otv_platts_usd_volume['2023-02-01' : '2023-03-31']

In [54]:
test['net_cashflow_from_operations'] = np.nan

/var/folders/t1/db5m68vn10v3148_nq_7r8b80000gn/T/ipykernel_1877/1338858615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['net_cashflow_from_operations'] = np.nan


In [55]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1462 entries, 2019-03-01 to 2023-02-01
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            1462 non-null   datetime64[ns]
 1   total_inflows                   1462 non-null   float64       
 2   customers_dds                   1462 non-null   int64         
 3   customers_eft                   1462 non-null   float64       
 4   t_&_s_collections               1462 non-null   float64       
 5   fx_sales                        1462 non-null   float64       
 6   other_operations                1462 non-null   float64       
 7   total_outflows                  1462 non-null   float64       
 8   tupras                          1462 non-null   float64       
 9   other_oil                       1462 non-null   float64       
 10  gas                             1462 non-null   float6

In [57]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 63 entries, 2023-02-01 to 2023-03-31
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            63 non-null     datetime64[ns]
 1   total_inflows                   63 non-null     float64       
 2   customers_dds                   63 non-null     int64         
 3   customers_eft                   63 non-null     float64       
 4   t_&_s_collections               63 non-null     float64       
 5   fx_sales                        63 non-null     float64       
 6   other_operations                63 non-null     float64       
 7   total_outflows                  63 non-null     float64       
 8   tupras                          63 non-null     float64       
 9   other_oil                       63 non-null     float64       
 10  gas                             63 non-null     float64 